In [5]:
import requests
import json
import datetime

api_key = "HkkbXgecZ_o88llpuTZ3hNvVjQdGyAhyQlJU8N9X9SfX6ZaS4G98q9nFE1B876xxnQnuLFdJXPr2pnsvtdq2krMpE17fskO8bIq6nwh451SAY9YDP2rTZwTxuRwrZXYx"
url = "https://api.yelp.com/v3/businesses/search"
headers = {
    "Authorization": f"Bearer {api_key}"
}

cuisine_types = ["chinese", "italian", "mexican", "indian", "japanese","french","thai"]
desired_count = 5000
restaurants = []

for cuisine in cuisine_types:
    offset = 0
    limit = 50

    while len(restaurants) < desired_count and offset < 1000:  # 1000 results per cuisine
        params = {
            "location": "Manhattan",
            "term": f"{cuisine} restaurants",
            "limit": limit,
            "offset": offset
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()

        for restaurant in data["businesses"]:
            if restaurant["id"] not in [r["Business ID"] for r in restaurants]:
                # Generate a unique timestamp with milliseconds
                current_time = datetime.datetime.now()
                unique_timestamp = current_time.timestamp() + current_time.microsecond / 1e6
                restaurant_data = {
                    "Business ID": restaurant["id"],
                    "Name": restaurant["name"],
                    "Address": restaurant["location"]["address1"],
                    "Coordinates": {
                        "Latitude": str(restaurant["coordinates"]["latitude"]),
                        "Longitude": str(restaurant["coordinates"]["longitude"]),
                    },
                    "Number of Reviews": str(restaurant["review_count"]),
                    "Rating": str(restaurant["rating"]),
                    "Zip Code": str(restaurant["location"]["zip_code"]),
                    'CuisineType': {'S': cuisine},  # Add the cuisine type to the item
                    'InsertedAtTimestamp': {'N': str(unique_timestamp)}
                }
                restaurants.append(restaurant_data)
                with open("data.json", 'a') as file:
                  file.write('{"index": {"_index": "restaurants", "_id": ' + str(restaurant_data['Business ID'] + '}}\n'))
                  file.write('{"Restaurant": ' + str(restaurant_data['Business ID']) + ', ' + '"cuisine": ' + cuisine + '}\n')

        offset += limit

        if len(restaurants) >= desired_count:
            break

# 'restaurants' list contains 5000 unique restaurants' data

# Write the 'restaurants' data to a JSON file
with open("restaurants_data.json", "w") as json_file:
    json.dump(restaurants, json_file, indent=4)



print("Done")


Done
